## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 24th Dec. 1st


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-27-2024,Condo/Co-op,1610 NW N River Dr #212,Miami,FL,33125.0,230000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1610-NW-North-...,MARMLS,A11679183,N,Y,25.786065,-80.221867


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
55,PAST SALE,2024-12-02,Condo/Co-op,800 West Ave #824,Miami Beach,FL,33139.0,260000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/800-West...,MARMLS,A11644231,N,Y,25.778011,-80.142466
69,PAST SALE,2024-12-01,Condo/Co-op,9195 Collins Ave #413,Surfside,FL,33154.0,410000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9195-Collin...,MARMLS,A11564873,N,Y,25.879678,-80.121703
22,PAST SALE,2024-11-30,Condo/Co-op,12105 NE 11th Pl #205,Biscayne Park,FL,33161.0,239000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/12105-NE...,MARMLS,A11644383,N,Y,25.887286,-80.174494
21,PAST SALE,2024-11-29,Condo/Co-op,3301 NE 1st Ave Unit H1907,Miami,FL,33137.0,740000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3301-NE-1st-Av...,MARMLS,A11670585,N,Y,25.807789,-80.192301
11,PAST SALE,2024-11-29,Condo/Co-op,7785 SW 86th St Unit E-424,Miami,FL,33143.0,259000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7785-SW-86th-S...,MARMLS,A11677288,N,Y,25.691092,-80.319333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,PAST SALE,2024-11-25,Condo/Co-op,255 Galen Dr Unit 3L,Key Biscayne,FL,33149.0,637000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/255-Gal...,MARMLS,A11683323,N,Y,25.691281,-80.160708
17,PAST SALE,2024-11-25,Condo/Co-op,3001 NE 185th St #416,Aventura,FL,33180.0,549000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3001-NE-185th-...,MARMLS,A11666424,N,Y,25.947558,-80.140242
18,PAST SALE,2024-11-25,Condo/Co-op,5201 Geneva Way #213,Doral,FL,33166.0,255000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/5201-Geneva-Wa...,MARMLS,A11673656,N,Y,25.818567,-80.328116
20,PAST SALE,2024-11-25,Condo/Co-op,2630 SW 28th St #41,Miami,FL,33133.0,325000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2630-SW-28th-S...,MARMLS,A11675090,N,Y,25.739008,-80.236921


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

83


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$651,882


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$465


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$54,106,167


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
113
Input Previous Week Condo Average Sales Price:
688002
Input Previous Week Condo Average PSF:
546
Input Previous Week Condo Sales total (ex: 198_000_000)
77_700_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-26,Condo/Co-op,10203 Collins Ave #1702,Bal Harbour,FL,33154.0,10875000.0,3.0,4.5,OCEANA BAL HARBOUR CONDO,3592.0,NaN,2016.0,NaN,3028.0,7277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11663105,N,Y,25.895557,-80.122903,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,Ethan Assouline,Douglas Elliman,Ruthie Assouline,Douglas Elliman,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Realty,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-26,Condo/Co-op,10203 Collins Ave #1702,Bal Harbour,FL,33154.0,10875000.0,3.0,4.5,OCEANA BAL HARBOUR CONDO,3592.0,NaN,2016.0,NaN,3028.0,7277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11663105,N,Y,25.895557,-80.122903,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,Ethan Assouline,Douglas Elliman,Ruthie Assouline,Douglas Elliman,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Realty,1,orange
1,PAST SALE,2024-11-26,Condo/Co-op,17875 Collins Ave #2906,Sunny Isles Beach,FL,33160.0,3850000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736.0,NaN,2006.0,NaN,1407.0,4500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11542695,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Kirill Kokarev,Broker One,None,None,Chad Carroll,"Compass Florida, LLC",None,None,2,blue
2,PAST SALE,2024-11-25,Condo/Co-op,200 Biscayne Boulevard Way #5202,Miami,FL,33131.0,2000000.0,3.0,3.5,EPIC WEST CONDO,2457.0,NaN,2008.0,NaN,814.0,4403.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/200-Biscayne-B...,MARMLS,A11533328,N,Y,25.770430,-80.189163,https://www.redfin.com/FL/Miami/200-Biscayne-B...,Alfredo Alvarez Cardona,Klerigi Realty LLC,Liana Doganiero,Klerigi Realty LLC,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,3,blue
3,PAST SALE,2024-11-27,Condo/Co-op,2600 Island Blvd #2004,Aventura,FL,33160.0,1900000.0,3.0,3.0,2600 ISL BLVD WILLIAMS IS,2580.0,NaN,1997.0,NaN,736.0,3545.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2600-Island...,MARMLS,A11673619,N,Y,25.939020,-80.133755,https://www.redfin.com/FL/Aventura/2600-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,4,blue
4,PAST SALE,2024-11-26,Condo/Co-op,2900 NE 7th Ave #1507,Miami,FL,33137.0,1585000.0,3.0,4.0,BISCAYNE BEACH CONDO,1868.0,NaN,2017.0,NaN,849.0,2560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11597856,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,5,blue
5,PAST SALE,2024-11-25,Condo/Co-op,17301 Biscayne Blvd #2106,North Miami Beach,FL,33160.0,1310000.0,3.0,3.5,MARINA PALMS RESIDENCES N,2158.0,NaN,2015.0,NaN,607.0,2210.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17301-Bisca...,MARMLS,A11677817,N,Y,25.935854,-80.148920,https://www.redfin.com/FL/Aventura/17301-Bisca...,Julia Nobre,"Julies Realty, LLC",None,None,Andy De Jesus,Best International Realty,None,None,6,blue
6,PAST SALE,2024-11-27,Condo/Co-op,1600 S Bayshore Ln Unit 9A,Miami,FL,33133.0,1200000.0,2.0,2.0,1600 SOUTH BAYSHORE LANE,1310.0,NaN,1970.0,NaN,916.0,1307.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,MARMLS,A11595040,N,Y,25.739483,-80.219286,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,Jill Penman,One Sotheby's International Realty,None,None,Ariana Bravo,Keller Williams Capital Realty,None,None,7,blue
7,PAST SALE,2024-11-27,Condo/Co-op,3700 Island Blvd Unit C207,Aventura,FL,33160.0,1100000.0,3.0,2.0,MEDITERRANEAN VILLAGE CON,1640.0,NaN,1989.0,NaN,671.0,4788.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3700-Island...,MARMLS,A11695178,N,Y,25.939679,-80.136044,https://www.redfin.com/FL/Aventura/3700-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,8,blue
8,PAST SALE,2024-11-26,Condo/Co-op,45 SW 9th St

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-26,Condo/Co-op,10203 Collins Ave #1702,Bal Harbour,FL,33154.0,10875000.0,3.0,4.5,OCEANA BAL HARBOUR CONDO,3592.0,NaN,2016.0,NaN,3028.0,7277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11663105,N,Y,25.895557,-80.122903,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,Ethan Assouline,Douglas Elliman,Ruthie Assouline,Douglas Elliman,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Realty,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_12022024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'OCEANA BAL HARBOUR CONDO at 10203 Collins Ave #1702 in Bal Harbour'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-26,Condo/Co-op,10203 Collins Ave #1702,Bal Harbour,FL,33154.0,10875000.0,3.0,4.5,OCEANA BAL HARBOUR CONDO,3592.0,NaN,2016.0,NaN,3028.0,7277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11663105,N,Y,25.895557,-80.122903,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,Ethan Assouline,Douglas Elliman,Ruthie Assouline,Douglas Elliman,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Realty,1,orange,OCEANA BAL HARBOUR CONDO at 10203 Collins Ave ...
1,PAST SALE,2024-11-26,Condo/Co-op,17875 Collins Ave #2906,Sunny Isles Beach,FL,33160.0,3850000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736.0,NaN,2006.0,NaN,1407.0,4500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11542695,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Kirill Kokarev,Broker One,None,None,Chad Carroll,"Compass Florida, LLC",None,None,2,blue,ACQUALINA OCEAN RESIDENCE at 17875 Collins Ave...
2,PAST SALE,2024-11-25,Condo/Co-op,200 Biscayne Boulevard Way #5202,Miami,FL,33131.0,2000000.0,3.0,3.5,EPIC WEST CONDO,2457.0,NaN,2008.0,NaN,814.0,4403.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/200-Biscayne-B...,MARMLS,A11533328,N,Y,25.770430,-80.189163,https://www.redfin.com/FL/Miami/200-Biscayne-B...,Alfredo Alvarez Cardona,Klerigi Realty LLC,Liana Doganiero,Klerigi Realty LLC,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,3,blue,EPIC WEST CONDO at 200 Biscayne Boulevard Way ...
3,PAST SALE,2024-11-27,Condo/Co-op,2600 Island Blvd #2004,Aventura,FL,33160.0,1900000.0,3.0,3.0,2600 ISL BLVD WILLIAMS IS,2580.0,NaN,1997.0,NaN,736.0,3545.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2600-Island...,MARMLS,A11673619,N,Y,25.939020,-80.133755,https://www.redfin.com/FL/Aventura/2600-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,4,blue,2600 ISL BLVD WILLIAMS IS at 2600 Island Blvd ...
4,PAST SALE,2024-11-26,Condo/Co-op,2900 NE 7th Ave #1507,Miami,FL,33137.0,1585000.0,3.0,4.0,BISCAYNE BEACH CONDO,1868.0,NaN,2017.0,NaN,849.0,2560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11597856,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,5,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #1507 ...
5,PAST SALE,2024-11-25,Condo/Co-op,17301 Biscayne Blvd #2106,North Miami Beach,FL,33160.0,1310000.0,3.0,3.5,MARINA PALMS RESIDENCES N,2158.0,NaN,2015.0,NaN,607.0,2210.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17301-Bisca...,MARMLS,A11677817,N,Y,25.935854,-80.148920,https://www.redfin.com/FL/Aventura/17301-Bisca...,Julia Nobre,"Julies Realty, LLC",None,None,Andy De Jesus,Best International Realty,None,None,6,blue,MARINA PALMS RESIDENCES N at 17301 Biscayne Bl...
6,PAST SALE,2024-11-27,Condo/Co-op,1600 S Bayshore Ln Unit 9A,Miami,FL,33133.0,1200000.0,2.0,2.0,1600 SOUTH BAYSHORE LANE,1310.0,NaN,1970.0,NaN,916.0,1307.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,MARMLS,A11595040,N,Y,25.739483,-80.219286,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,Jill Penman,One Sotheby's International Realty,None,None,Ariana Bravo,Keller Williams Capital Realty,None,None,7,blue,1600 SOUTH BAYSHORE LANE at 1600 S Bayshore Ln...
7,PAST SALE,2024-11-27,Condo/Co-op,3700 Island Blvd Unit C207,Aventura,FL,33160.0,1100000.0,3.0,2.0,MEDITERRANEA

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: OCEANA BAL HARBOUR CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,050,000M to $10,875,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Sunny Isles Beach, North Miami Beach, Aventura, Miami, Bal Harbour

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $54,106,167 million from Nov. 24th to Dec. 1st. The previous week, brokers closed 113 condo sales totaling $77,700,000.

Last week’s units sold for an average of $651,882, lower than the $688,002 average price from the previous week. The 

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-26,Condo/Co-op,10203 Collins Ave #1702,Bal Harbour,FL,33154.0,10875000.0,3.0,4.5,OCEANA BAL HARBOUR CONDO,3592.0,NaN,2016.0,NaN,3028.0,7277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11663105,N,Y,25.895557,-80.122903,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,Ethan Assouline,Douglas Elliman,Ruthie Assouline,Douglas Elliman,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Realty,1,orange,OCEANA BAL HARBOUR CONDO at 10203 Collins Ave ...
1,PAST SALE,2024-11-26,Condo/Co-op,17875 Collins Ave #2906,Sunny Isles Beach,FL,33160.0,3850000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736.0,NaN,2006.0,NaN,1407.0,4500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11542695,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Kirill Kokarev,Broker One,None,None,Chad Carroll,"Compass Florida, LLC",None,None,2,blue,ACQUALINA OCEAN RESIDENCE at 17875 Collins Ave...
2,PAST SALE,2024-11-25,Condo/Co-op,200 Biscayne Boulevard Way #5202,Miami,FL,33131.0,2000000.0,3.0,3.5,EPIC WEST CONDO,2457.0,NaN,2008.0,NaN,814.0,4403.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/200-Biscayne-B...,MARMLS,A11533328,N,Y,25.770430,-80.189163,https://www.redfin.com/FL/Miami/200-Biscayne-B...,Alfredo Alvarez Cardona,Klerigi Realty LLC,Liana Doganiero,Klerigi Realty LLC,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,3,blue,EPIC WEST CONDO at 200 Biscayne Boulevard Way ...
3,PAST SALE,2024-11-27,Condo/Co-op,2600 Island Blvd #2004,Aventura,FL,33160.0,1900000.0,3.0,3.0,2600 ISL BLVD WILLIAMS IS,2580.0,NaN,1997.0,NaN,736.0,3545.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2600-Island...,MARMLS,A11673619,N,Y,25.939020,-80.133755,https://www.redfin.com/FL/Aventura/2600-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,4,blue,2600 ISL BLVD WILLIAMS IS at 2600 Island Blvd ...
4,PAST SALE,2024-11-26,Condo/Co-op,2900 NE 7th Ave #1507,Miami,FL,33137.0,1585000.0,3.0,4.0,BISCAYNE BEACH CONDO,1868.0,NaN,2017.0,NaN,849.0,2560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11597856,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,5,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #1507 ...
5,PAST SALE,2024-11-25,Condo/Co-op,17301 Biscayne Blvd #2106,North Miami Beach,FL,33160.0,1310000.0,3.0,3.5,MARINA PALMS RESIDENCES N,2158.0,NaN,2015.0,NaN,607.0,2210.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17301-Bisca...,MARMLS,A11677817,N,Y,25.935854,-80.148920,https://www.redfin.com/FL/Aventura/17301-Bisca...,Julia Nobre,"Julies Realty, LLC",None,None,Andy De Jesus,Best International Realty,None,None,6,blue,MARINA PALMS RESIDENCES N at 17301 Biscayne Bl...
6,PAST SALE,2024-11-27,Condo/Co-op,1600 S Bayshore Ln Unit 9A,Miami,FL,33133.0,1200000.0,2.0,2.0,1600 SOUTH BAYSHORE LANE,1310.0,NaN,1970.0,NaN,916.0,1307.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,MARMLS,A11595040,N,Y,25.739483,-80.219286,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,Jill Penman,One Sotheby's International Realty,None,None,Ariana Bravo,Keller Williams Capital Realty,None,None,7,blue,1600 SOUTH BAYSHORE LANE at 1600 S Bayshore Ln...
7,PAST SALE,2024-11-27,Condo/Co-op,3700 Island Blvd Unit C207,Aventura,FL,33160.0,1100000.0,3.0,2.0,MEDITERRANEA

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: OCEANA BAL HARBOUR CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,050,000M to $10,875,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Sunny Isles Beach, North Miami Beach, Aventura, Miami, Bal Harbour

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $54,106,167 million from Nov. 24th to Dec. 1st. The previous week, brokers closed 113 condo sales totaling $77,700,000.

Last week’s units sold for an average of $651,882, lower than the $688,002 average price from the previous week. The 

In [48]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Bal-Harbour/10203-Collins-Ave-33154/unit-1702/home/56745853


In [49]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17875-Collins-Ave-33160/unit-2906/home/43376234


In [50]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Bal-Harbour/10175-Collins-Ave-33154/unit-1506/home/42903004


In [51]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/45-SW-9th-St-33130/unit-4106/home/79134844


In [52]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Bal-Harbour/10203-Collins-Ave-33154/unit-1702/home/56745853


In [53]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Bal-Harbour/10175-Collins-Ave-33154/unit-1506/home/42903004


In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-26,Condo/Co-op,10203 Collins Ave #1702,Bal Harbour,FL,33154.0,10875000.0,3.0,4.5,OCEANA BAL HARBOUR CONDO,3592.0,NaN,2016.0,NaN,3028.0,7277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11663105,N,Y,25.895557,-80.122903,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,Ethan Assouline,Douglas Elliman,Ruthie Assouline,Douglas Elliman,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Realty,1,orange,OCEANA BAL HARBOUR CONDO at 10203 Collins Ave ...
1,PAST SALE,2024-11-26,Condo/Co-op,17875 Collins Ave #2906,Sunny Isles Beach,FL,33160.0,3850000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736.0,NaN,2006.0,NaN,1407.0,4500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11542695,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Kirill Kokarev,Broker One,None,None,Chad Carroll,"Compass Florida, LLC",None,None,2,blue,ACQUALINA OCEAN RESIDENCE at 17875 Collins Ave...
2,PAST SALE,2024-11-25,Condo/Co-op,200 Biscayne Boulevard Way #5202,Miami,FL,33131.0,2000000.0,3.0,3.5,EPIC WEST CONDO,2457.0,NaN,2008.0,NaN,814.0,4403.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/200-Biscayne-B...,MARMLS,A11533328,N,Y,25.770430,-80.189163,https://www.redfin.com/FL/Miami/200-Biscayne-B...,Alfredo Alvarez Cardona,Klerigi Realty LLC,Liana Doganiero,Klerigi Realty LLC,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,3,blue,EPIC WEST CONDO at 200 Biscayne Boulevard Way ...
3,PAST SALE,2024-11-27,Condo/Co-op,2600 Island Blvd #2004,Aventura,FL,33160.0,1900000.0,3.0,3.0,2600 ISL BLVD WILLIAMS IS,2580.0,NaN,1997.0,NaN,736.0,3545.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2600-Island...,MARMLS,A11673619,N,Y,25.939020,-80.133755,https://www.redfin.com/FL/Aventura/2600-Island...,Joel Matus,"Williams Island Realty, LLC",None,None,Joel Matus,"Williams Island Realty, LLC",None,None,4,blue,2600 ISL BLVD WILLIAMS IS at 2600 Island Blvd ...
4,PAST SALE,2024-11-26,Condo/Co-op,2900 NE 7th Ave #1507,Miami,FL,33137.0,1585000.0,3.0,4.0,BISCAYNE BEACH CONDO,1868.0,NaN,2017.0,NaN,849.0,2560.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11597856,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,Bradley Piper,United Realty Group Inc,Cindy Binder,United Realty Group Inc,5,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #1507 ...
5,PAST SALE,2024-11-25,Condo/Co-op,17301 Biscayne Blvd #2106,North Miami Beach,FL,33160.0,1310000.0,3.0,3.5,MARINA PALMS RESIDENCES N,2158.0,NaN,2015.0,NaN,607.0,2210.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/17301-Bisca...,MARMLS,A11677817,N,Y,25.935854,-80.148920,https://www.redfin.com/FL/Aventura/17301-Bisca...,Julia Nobre,"Julies Realty, LLC",None,None,Andy De Jesus,Best International Realty,None,None,6,blue,MARINA PALMS RESIDENCES N at 17301 Biscayne Bl...
6,PAST SALE,2024-11-27,Condo/Co-op,1600 S Bayshore Ln Unit 9A,Miami,FL,33133.0,1200000.0,2.0,2.0,1600 SOUTH BAYSHORE LANE,1310.0,NaN,1970.0,NaN,916.0,1307.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,MARMLS,A11595040,N,Y,25.739483,-80.219286,https://www.redfin.com/FL/Miami/1600-S-Bayshor...,Jill Penman,One Sotheby's International Realty,None,None,Ariana Bravo,Keller Williams Capital Realty,None,None,7,blue,1600 SOUTH BAYSHORE LANE at 1600 S Bayshore Ln...
7,PAST SALE,2024-11-27,Condo/Co-op,3700 Island Blvd Unit C207,Aventura,FL,33160.0,1100000.0,3.0,2.0,MEDITERRANEA

## Time on Market Calculator

In [66]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 3, 1) ## List (Earlier) date
date2 = datetime(2024, 11, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

270


## Clean CSV for Datawrapper Chart

In [56]:
chart_df = df_top_ten

In [57]:
chart_df = chart_df.fillna(" ")

In [58]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [59]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [60]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [61]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [62]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [63]:
csv_date_string = today.strftime("%m_%d_%Y")

In [64]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [65]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-26,10203 Collins Ave #1702 in Bal Harbour,"$10,875,000",3,4.5,"3,592",2016,"$3,028",MARMLS,A11663105,Ethan Assouline Douglas Elliman Ruthie Assouli...,Anna Sherrill One Sotheby's International Re A...
1,2024-11-26,17875 Collins Ave #2906 in Sunny Isles Beach,"$3,850,000",4,4.0,"2,736",2006,"$1,407",MARMLS,A11542695,Kirill Kokarev Broker One,"Chad Carroll Compass Florida, LLC"
2,2024-11-25,200 Biscayne Boulevard Way #5202 in Miami,"$2,000,000",3,3.5,"2,457",2008,$814,MARMLS,A11533328,Alfredo Alvarez Cardona Klerigi Realty LLC Lia...,NON-MLS MEMBER MAR NON MLS MEMBER
3,2024-11-27,2600 Island Blvd #2004 in Aventura,"$1,900,000",3,3.0,"2,580",1997,$736,MARMLS,A11673619,"Joel Matus Williams Island Realty, LLC","Joel Matus Williams Island Realty, LLC"
4,2024-11-26,2900 NE 7th Ave #1507 in Miami,"$1,585,000",3,4.0,"1,868",2017,$849,MARMLS,A11597856,Bradley Piper United Realty Group Inc Cindy Bi...,Bradley Piper United Realty Group Inc Cindy Bi...
5,2024-11-25,17301 Biscayne Blvd #2106 in North Miami Beach,"$1,310,000",3,3.5,"2,158",2015,$607,MARMLS,A11677817,"Julia Nobre Julies Realty, LLC",Andy De Jesus Best International Realty
6,2024-11-27,1600 S Bayshore Ln Unit 9A in Miami,"$1,200,000",2,2.0,"1,310",1970,$916,MARMLS,A11595040,Jill Penman One Sotheby's International Realty...,Ariana Bravo Keller Williams Capital Realty
7,2024-11-27,3700 Island Blvd Unit C207 in Aventura,"$1,100,000",3,2.0,"1,640",1989,$671,MARMLS,A11695178,"Joel Matus Williams Island Realty, LLC","Joel Matus Williams Island Realty, LLC"
8,2024-11-26,45 SW 9th St #4106 in Miami,"$1,100,000",3,3.0,"1,364",2017,$806,MARMLS,A11679743,Patricia Millan Million Realty LLC Mikal Ramir...,"Luis Uribe LPT Realty, LLC"
9,2024-11-25,10175 Collins Ave #1506 in Bal Harbour,"$1,050,000",2,2.5,"2,018",1982,$520,MARMLS,A11589525,Shawn Frechette One Sotheby's International Re...,Shawn Frechette One Sotheby's International Re...
